In [1]:
# Extract and save data from database in AWS
import db_utils as dbu
# Get descritve informarion from the dataframe
import extract_info as extract
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Creates plots for data
import plot as plo
# Tranform dataframe and columns
import pre_processing as prep   
from statsmodels.graphics.gofplots import qqplot
import seaborn as sns
import statsmodels.formula.api as smf

# DataBase

In this section the RDSDatabaseConnector class from db_utils.py was used to connect to AWS RDS and download the data into the local machine.

In [2]:
# Read the AWS RDS credentials from yaml file
db_credentials = dbu.read_credentials("/Users/ChAre/OneDrive/Desktop/aicore/EDA_finance/credentials.yaml")

In [3]:
# Initialise RDSDatabaseConnector class
db_connector = dbu.RDSDatabaseConnector(db_credentials)

A connection was made, the data extracted and saved as a csv file into the local machine using save_data() method. This class method also called the methods for connecting and extracting the data.

In [4]:
# Connect to the AWS RDS, extract and save the data
#db_connector.save_data()

Then the data was loaded into python as a dataframe.

In [5]:
# Load data into pyhton
df_unclean = db_connector.load_data("./loan_payments")

# Data preprocessing

In this section the dataframe colums were transformed to their appropriate data type, missing values were removed or imputed, the data was transformed to correct skeweness, outlier were removed and overly correlated columns were dropped.

#### Transforming columns

In [6]:
# Inspect the columns data types
df_unclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  int64  
 1   member_id                    54231 non-null  int64  
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

Not all columns were converted to their correct data type especially those who are categorical or dates type data. Therefore the columns were transformed using the methods in the DataTransform class in the pre_processing.py file.

In [7]:
# Create instance of preprocessing of DataTransform class for preprocessing
df_prep = prep.DataTransform(df_unclean)

In [8]:
# Transform columns into the categorical data type
df_prep.category_transform(["grade", "sub_grade", "home_ownership", "verification_status", "loan_status", "payment_plan", "purpose", "application_type", "employment_length", "policy_code", "term"])

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36 months,7.49,248.82,A,A4,...,0.00,0.00,Jan-2022,248.82,Feb-2022,Jan-2022,0.0,5.0,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36 months,6.99,407.52,A,A3,...,0.00,0.00,Jan-2022,407.52,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36 months,7.49,497.63,A,A4,...,0.00,0.00,Oct-2021,12850.16,NaN,Oct-2021,0.0,NaN,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36 months,14.31,514.93,C,C4,...,0.00,0.00,Jun-2021,13899.67,NaN,Jun-2021,0.0,NaN,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36 months,6.03,456.54,A,A1,...,0.00,0.00,Jan-2022,456.54,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54226,76597,76583,5000,5000.0,1775.0,36 months,9.01,159.03,B,B2,...,0.00,0.00,Jul-2016,160.61,NaN,Jul-2016,NaN,NaN,1,INDIVIDUAL
54227,142608,74724,5000,5000.0,2350.0,36 months,10.59,162.73,C,C2,...,0.00,0.00,Oct-2016,490.01,NaN,Sep-2016,0.0,NaN,1,INDIVIDUAL
54228,117045,70978,3500,3500.0,2225.0,36 months,7.43,108.77,A,A2,...,0.00,0.00,Sep-2016,110.58,Sep-2016,May-2013,NaN,NaN,1,INDIVIDUAL
54229,88854,70699,5000,5000.0,225.0,36 months,7.43,155.38,A,A2,...,0.00,0.00,Mar-2014,0.00,NaN,May-2013,NaN,NaN,1,INDIVIDUAL


In [9]:
# Convert columns to date type data
df_prep.date_transform(["issue_date", "earliest_credit_line", "last_payment_date", "next_payment_date", "last_credit_pull_date"])

Although the term column had has the "month" next to the actual value, this data would be more useful as numeric since this colums tell us the number of motnhly payment to the loan, which could  be use for calculations. However, first it was converted to string to remove "month" and then was converted to the integer type and the column name was updated to "term_in_months" to undertand the context of the numeric data.

In [ ]:
# Convert column values into strings
df_prep.string_transform(["term"])

# Remove unwanted characters from the column values
df_prep.remove_characters("term", ["months"])

# Transform column values to numeric
df_prep.numeric_transform(["term"])

#Rename column to reflect the unit for the values
df_prep.rename_column("term", "term_in_months")

#### Removing/imputing missing values

In [ ]:
# Create DataFrameInfo instance to extract useful information from the dataframe.
df_inf = extract.DataFrameInfo(df_prep.dataframe)

In [ ]:
# Check the data types of the columns after the transformations.
df_inf.check_columns_type()

The percentage of null per column of the dataframe was generated, to visualise the proportion on missing values across the whole data and to undertand which columns will need further pre_processing.

In [ ]:
# Generate a count/percentage count of NULL values in each column
df_inf.count_null(percentage=True)

Base on the percentage of missing data I decided to:

Drop:
 - mths_since_last_delinq:        57.2% missing values
 - mths_since_last_record:        88.6% missing values
 - mths_since_last_major_derog:   86.2% missing values
 
 Based on the large proprotion of missing data. Although these data could be imputed if needed it would most likely require machine learning and therefore an extended period of time. 

Impute:
- funded_amount:                   5.5% missing values
- term_in_months:                  8.8% missing values
- int_rate:                        9.5% missing values
- employment_length:               3.9% missing values
- last_payment_date:               0.1% missing values
- last_credit_pull_date:           0.013% missing values
- collections_12_mths_ex_med:      0.094% missing values
- next_payment_date:               86.2%  missing values

Although the next_payment_date column has 86.2% of missing values, it may be useful to project the profit for the following month but I chose to impute it. I also prioritise imputing values rather than dropping rows since it  will reduce the data in other column decreasing the accuracy of imputation for other values  and inference made from the data.

In [ ]:
# Dropping columns
df_prep.remove_columns(["mths_since_last_delinq", "mths_since_last_record", "mths_since_last_major_derog"])

Normality test were carried on numerical and date type columns to undertand their distribution to select wether the mean, media or mode is the most appropriate method for imputation.Therefore, the Plotter class was used to visualise the data.

In [ ]:
# Initiate instance
plots = plo.Plotter(df_prep.dataframe)

##### Funded amount

In [ ]:
# Normality test
df_inf.norm_test("funded_amount")
# Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column="funded_amount",
                title= "The total amount committed to the loan at that point in time",
                xlabel="Amount in pounds",
                ylabel="Frequency")
# QQplot
plots.plot_qq(dataframe = df_inf.dataframe,
              column = "funded_amount")


Although the normality test had a significant P-value (p<0.05), both the histogram and QQ-plot showed that the data has a slight positive skew. Therefore the median will be used to impute missing values.

In [ ]:
# Imputing Values

# Get the mendian
df_inf.descriptive_stats(selected_column=["funded_amount"])

# Replace missing values with the median
df_prep.replace_null("funded_amount", 12000.0)


##### Term in months

In [ ]:
# Normality test
df_inf.norm_test("term_in_months")

# Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column="term_in_months",
                title= "Distribution of the total monthly payments for the loan",
                xlabel="Total monthly payments",
                ylabel="Frequency")

# QQplot
plots.plot_qq(dataframe= plots.dataframe,
              column="term_in_months")

Both the histogram and QQ plot showed no normal distribution although the normality test's p value was significant. Also the histrogram could be interpreted as the data having positive skew. Therefore I will impute using the median.

In [ ]:
# Impute values of the "term_in_months" column

# Get the mendian
df_inf.descriptive_stats(selected_column=["term_in_months"])
# replace null values
df_prep.replace_null("term_in_months", 36.0)

##### Interest rate

In [ ]:
# Normality test
df_inf.norm_test("int_rate")

# Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column= "int_rate",
                title= "Annual (APR) interest rate of the loan",
                xlabel="Annual inetrest rates",
                ylabel="Frequency")


# QQplot
plots.plot_qq(dataframe= plots.dataframe,
              column="int_rate")

Although the normality test showed a significant p value, both the histogram and QQ-plot displayed sligthly positive skewness of the data. Therefore i will use the median to impute missing values.

In [ ]:
# Impute values 

# Get the mendian
df_inf.descriptive_stats(selected_column=["int_rate"])

# replace null values
df_prep.replace_null("int_rate", 13.16)

##### Lats payment date

In [ ]:
# Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column= "last_payment_date",
                title= "Date on which last month payment was received",
                xlabel="Dates",
                ylabel="Frequency")

The histogram showed prominent negative skew of the data but the QQplot and normality test could not be used to to conflict with the date type data. Thereofre, missing values will be imputed using the median.

In [ ]:
# Impute values 

# Get the mendian
df_inf.descriptive_stats( selected_column=["last_payment_date"])

# replace null values
df_prep.replace_null("last_payment_date",  pd.to_datetime('2021-04-01 00:00:00'))

##### Last credit pull date

In [ ]:
#Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column= "last_credit_pull_date",
                title= "Distribution of the most recent month that the loan company pulled credit for this loan",
                xlabel="Dates",
                ylabel="Frequency")

The histogram showed prominent negative skewness of the data. Therefore the median will be used to impute missing values.

In [ ]:
# Impute values

# Get the mendian
df_inf.descriptive_stats( selected_column=["last_credit_pull_date"])

# replace null values
df_prep.replace_null("last_credit_pull_date",  pd.to_datetime('2022-01-01 00:00:00'))

#####  Collections 12 months

In [ ]:
# Normality test
df_inf.norm_test("collections_12_mths_ex_med")

# Historgram
plots.hist_plot(dataframe= plots.dataframe,
                column="collections_12_mths_ex_med",
                title= "Distribution of the Number of collections in 12 months excluding medical collections",
                xlabel="Number of collections",
                ylabel="Frequency")

# QQplot
plots.plot_qq(dataframe= plots.dataframe,
              column="collections_12_mths_ex_med")

Although the normality test p value is significant the  histogram and qq plot, showed a non normal distribution. However, this might be due to insufficient data but for this the data was treated as having a positive skewed and so the median was used to inpute the missing values.

In [ ]:
#Impute values

# Get the median
df_inf.descriptive_stats(selected_column=["collections_12_mths_ex_med"])
# replace null values
df_prep.replace_null("collections_12_mths_ex_med", 0.0)

##### Next payment date

In [ ]:

# Histogram
plots.hist_plot(dataframe= plots.dataframe,
                column="next_payment_date",
                title= "Distribution of the next scheduled payment date",
                xlabel='Next schedualed payment date',
                ylabel="Frequency")

The histogram showed negative skew of the data. Therefore missing values were imputed based on the median. The QQplot and normality test coudl not be done due to conflicts with the date data type. I acknowlwedge that since there is a large protion of data missing in this column imputing missing values based on median, mean or mode is not the best choice. However, the alternative would be using machine learning but the distance between null values an actual values in the column had to be taken into account since imputing them based their nearest neighbour would not be relaible in my opinion if there is a large gap between actual values and missing values.

In [ ]:
# Get the mendian
df_inf.descriptive_stats( selected_column=["next_payment_date"])

# replace null values
df_prep.replace_null("next_payment_date",  pd.to_datetime('2022-02-01 00:00:00'))

##### Employment length

In [ ]:
# Histogram
plots.hist_plot(dataframe= plots.dataframe,
                column="employment_length",
                title= "Duration in which the client has been working",
                xlabel="Years of employment",
                ylabel="Frequency")

The histogram showed positive skew of the data. However, since the data is categorical the mode was used.

In [ ]:
# Impute values
# Get the mode
df_prep.dataframe["employment_length"].mode()
# replace null values
df_prep.replace_null("employment_length", "10+ years")


In [ ]:
plots.missing_values_plot()

The dataframe wiht the imputation will be save locally and be used for further sections with require transformation. This action was taken to prevent the transformations affecting the actual values of the data that will be used for the analysis sections. 

In [ ]:
#df_prep.dataframe.to_csv("impute_loan_payments", index=False)

# Dealing with skewness

In this section the skewness of the numeric columns was measure, if their skewness was bigger than or equla to 0.5 they were considered skewed. Their skewness were corrected using a combination of histogram to visualise the skewness, transformations (log, Box-Cox, Yeo-Johnson) to alter the distribution shape and histogram to select the most effective transformation. In some cases the Box-Cox transformation was omitted due to the presence of zero and negative numbers.



In [ ]:
# Create a new intance with the impute dataframe to test for skewness
df_inf_imp = extract.DataFrameInfo(df_prep.dataframe)

In [ ]:
# Check skewness
df_inf_imp.skew_check()

The following columns were classified as skewed and therefore will be transformed.

- loan_amount:        0.81 
- funded_amount:      0.87
- funded_amount_inv:  0.81 
- term_in_moths:      1.15
- instalmet:          1.0
- annual_inc:         8.71  
- delinq_2yrs:        5.37 
- inq_last_6mths:     3.25
- open_accounts:      1.06
- total_accounts:     0.78
- out_prncp:          2.36 
- out_prncp_inv:      2.36
- total_payment:      1.27 
- total_payment_inv:  1.26 
- total_rec_prncp:    1.26
- total_rec_int:      2.2 
- total_rec_late_fee: 13.18 
- recoveries:         14.59 
- collection_recovery_fee: 27.64
- last_payment_amount: 2.5
- collections_12_mths_ex_med: 20.26

##### Loan amount

In [ ]:
# Visualise the skew
plots.hist_plot(dataframe= df_prep.dataframe,
                column="loan_amount",
                title= "Amount of loan the applicant received",
                xlabel="Amount in pounds",
                ylabel="Frequency")

- Positive skew

In [ ]:
# Apply log transformation.
loan_log_transform = df_prep.skew_transform(df_prep.dataframe["loan_amount"], transformation="log")
loan_BC_transform = df_prep.skew_transform(df_prep.dataframe["loan_amount"], transformation="BC")
loan_YJ_transform = df_prep.skew_transform(df_prep.dataframe["loan_amount"], transformation="YJ")

# Histogram for log transform data
plots.hist_plot(dataframe= loan_log_transform,
                column="log",
                title="Distribution of log transformed loan amount",
                xlabel="Amount in pounds",
                ylabel="Frequency")


# Histogram for Box-Cox transform data
plots.hist_plot(dataframe= loan_BC_transform,
                column="Box-Cox",
                title="Distribution of Box-Cox transformed loan amount",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Histogram for Yeo-Johnson transform data
plots.hist_plot(dataframe= loan_YJ_transform,
                column="Yeo-Johnson",
                title="Distribution of Yeo-Johnson transformed loan amount",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:

# QQplot
plots.plot_qq(dataframe= loan_log_transform,
              column="log")

plots.plot_qq(dataframe= loan_BC_transform,
              column="Box-Cox")

plots.plot_qq(dataframe= loan_YJ_transform,
              column="Yeo-Johnson")

- Box-Cox and Yeo-Johnson trasnformation produced the greater effect in the distribution.

- Applied Box-Cox since there was not presence of negative or zero values.

In [ ]:
# Applied transformation
df_prep.dataframe["loan_amount"] = loan_log_transform["log"]

##### Funded amount

In [ ]:
# Histogram
plots.hist_plot(dataframe= df_prep.dataframe,
                column="funded_amount",
                title= "The total amount committed to the loan at that point in time",
                xlabel="Amount in pounds",
                ylabel="Frequency")

 - Positive skew data.

In [ ]:
# Apply log transformation.
funded_am_log_transform = df_prep.skew_transform(df_prep.dataframe["funded_amount"], transformation="log")
funded_am_BC_transform = df_prep.skew_transform(df_prep.dataframe["funded_amount"], transformation="BC")
funded_am_YJ_transform = df_prep.skew_transform(df_prep.dataframe["funded_amount"], transformation="YJ")

# Histogram log
plots.hist_plot(dataframe= funded_am_log_transform,
                column="log",
                title= " Log transformed total amount committed to the loan at that point in time",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Histogram Box-Cox
plots.hist_plot(dataframe= funded_am_BC_transform,
                column="Box-Cox",
                title= "Box-Cox transformed total amount committed to the loan at that point in time",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Histogram Yeo-Johnson
plots.hist_plot(dataframe= funded_am_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson transformed total amount committed to the loan at that point in time",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=funded_am_log_transform,
              column="log")
plots.plot_qq(dataframe=funded_am_BC_transform,
              column="Box-Cox")
plots.plot_qq(dataframe=funded_am_YJ_transform,
              column="Yeo-Johnson")

- Box-Cox and Yeo-Johnson produce the greates effect
- Box-Cox was chosen since the data dosen't have any zero or nagative values.

In [ ]:
# Skew transformation
df_prep.dataframe["funded_amount"] = funded_am_BC_transform

##### Funded amount inv

In [ ]:
# Visualise the skew of funded_amount_inv
plt.figure(figsize=(10, 6))
sns.histplot(data=df_prep.dataframe, x="funded_amount_inv", bins= 10)
plt.title('Distribution of the total amount committed by the insvestors for that loan at that point in time')
plt.xlabel('Total amount comitted in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y")
plt.show()

- Positive skew data

In [ ]:
# Apply log transformation.
funded_log_transform = df_prep.skew_transform(df_prep.dataframe["funded_amount_inv"], transformation="log")
funded_YJ_transform = df_prep.skew_transform(df_prep.dataframe["funded_amount_inv"], transformation="YJ")

# Histogram for log transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=funded_log_transform, x="log", bins= 10)
plt.title('Log transformed of the total amount committed by investors for that loan at that point in time')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

# Histogram for Yeo-Johnson transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=funded_YJ_transform, x="Yeo-Johnson", bins= 10)
plt.title('Yeo-Johnson transformed of the total amount committed by investors for that loan at that point in time')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y")
plt.show()

In [ ]:
# QQplot
qq_plot = qqplot(funded_log_transform["log"], scale=1 ,line='q')
qq_plot = qqplot(funded_YJ_transform["Yeo-Johnson"], scale=1 ,line='q')
pyplot.show()

- Yeo-Johnson trasnformation produce the greater effect and therefore was used.

In [ ]:
df_prep.dataframe["funded_amount_inv"] = funded_YJ_transform 

##### Instalment

In [ ]:
# Histogram of original data
plt.figure(figsize=(10, 6))
sns.histplot(data=df_prep.dataframe, x="instalment", bins= 10)
plt.title('Monthly payment owned by the borrower including interest')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

- Positive skew data.

In [ ]:
# Apply log transformation.
instal_log_transform = df_prep.skew_transform(df_prep.dataframe["instalment"], transformation="log")
instal_BC_transform = df_prep.skew_transform(df_prep.dataframe["instalment"], transformation="BC")
instal_YJ_transform = df_prep.skew_transform(df_prep.dataframe["instalment"], transformation="YJ")

# Histogram for log transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=instal_log_transform, x="log", bins= 10)
plt.title('Log transformed of the monthly payment owned by the borrower including interest')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

# Histogram for log transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=instal_BC_transform, x="Box-Cox", bins= 10)
plt.title('Cox-Box transformed of the monthly payment owned by the borrower including interest')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

# Histogram for Yeo-Johnson transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=instal_YJ_transform, x="Yeo-Johnson", bins= 10)
plt.title('Yeo-Johnson transformed of the monthly payment owned by the borrower including interest')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y")
plt.show()

In [ ]:

# QQplot
qq_plot = qqplot(instal_log_transform["log"], scale=1 ,line='q')
qq_plot = qqplot(instal_BC_transform["Box-Cox"], scale=1 ,line='q')
qq_plot = qqplot(instal_YJ_transform["Yeo-Johnson"], scale=1 ,line='q')
pyplot.show()

- Box-Cox and Yeo-Johnson produced the same results but since there were no zero or negative  values, Box-Cox was used.

In [ ]:
df_prep.dataframe["instalment"]  = instal_BC_transform

##### Annual inc

In [ ]:
# Original Data
plt.figure(figsize=(10, 6))
sns.histplot(data=df_prep.dataframe, x="annual_inc", bins= 5)
plt.title('annual income of the borrower')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

The histogram shows left skew of the data.

In [ ]:
# Apply log transformation.
an_in_log_transform = df_prep.skew_transform(df_prep.dataframe["annual_inc"], transformation="log")
an_in_BC_transform = df_prep.skew_transform(df_prep.dataframe["annual_inc"], transformation="BC")
an_in_YJ_transform = df_prep.skew_transform(df_prep.dataframe["annual_inc"], transformation="YJ")

# Histogram for log transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=an_in_log_transform, x="log", bins= 10, kde=True)
plt.title('Log transformed of the annual income of the borrower')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

# Histogram for log transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=an_in_BC_transform, x="Box-Cox", bins= 10, kde=True)
plt.title('Cox-Box transformed of the annual income of the borrower')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y") 
plt.show()

# Histogram for Yeo-Johnson transform data
plt.figure(figsize=(10, 6))
sns.histplot(data=an_in_YJ_transform, x="Yeo-Johnson", bins= 10, kde=True)
plt.title('Yeo-Johnson transformed of the annual income of the borrower')
plt.xlabel('Amount in pounds')
plt.ylabel('Frequency')
plt.grid(True, axis="y")
plt.show()

In [ ]:
# QQplot
qq_plot = qqplot(an_in_log_transform["log"], scale=1 ,line='q')
qq_plot = qqplot(an_in_BC_transform["Box-Cox"], scale=1 ,line='q')
qq_plot = qqplot(an_in_YJ_transform["Yeo-Johnson"], scale=1 ,line='q')
pyplot.show()

Histograms an qq plost showed that any of the three used transfomations can be use and produce very similar chnages in distribution but seems that the log transformation produces the most normal distribution. therefore this will be used.

In [ ]:
df_prep.dataframe["annual_inc"]  = an_in_log_transform 

##### Total payment

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="total_payment", 
                title= "Payments received to date for total amount funded", 
                xlabel="Amount in pounds",
                ylabel="Frequency")

This histogram shows positve/ right skew of the data.

In [ ]:
# Apply log transformation.
total_p_in_log_transform = df_prep.skew_transform(df_prep.dataframe["total_payment"], transformation="log")
total_p_YJ_transform = df_prep.skew_transform(df_prep.dataframe["total_payment"], transformation="YJ")

# Plot log transformation
plots.hist_plot(dataframe = total_p_in_log_transform,
                column="log", 
                title= "Log transformed payments received to date for the total amount funded", 
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Plot Box-Cox transformation
plots.hist_plot(dataframe = total_p_YJ_transform,
                column="Yeo-Johnson", 
                title= "Yeo-Johnson transformed Payments received to date for the total amount funded", 
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=total_p_in_log_transform,
              column="log")
plots.plot_qq(dataframe=total_p_YJ_transform,
              column="Yeo-Johnson")

Histograms and qq plots showed that the Yeo-Johnson trasnformation ahs the greates effect on the data. Therefore we will applied that transformation.

In [ ]:
df_prep.dataframe["total_payment"] = total_p_YJ_transform


##### Total rec int

In [ ]:
# Original data
plots.hist_plot(dataframe= df_prep.dataframe,
                column="total_rec_int",
                title= "Interest received to date",
                xlabel="Amount of inetrest",
                ylabel="Frequency")

In [ ]:
# Transformations
total_rec_log_transform = df_prep.skew_transform(df_prep.dataframe["total_rec_int"], transformation="log")

total_rec_YJ_transform = df_prep.skew_transform(df_prep.dataframe["total_rec_int"], transformation="YJ")

# Histograms
plots.hist_plot(dataframe= total_rec_log_transform,
                column="log",
                title= "Log transformed interest received to date",
                xlabel="Amount of inetrest",
                ylabel="Frequency")

plots.hist_plot(dataframe= total_rec_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson transformed interest received to date",
                xlabel="Amount of inetrest",
                ylabel="Frequency")

In [ ]:
# QQ_plots
plots.plot_qq(dataframe=total_rec_log_transform ,
              column="log")
plots.plot_qq(dataframe=total_rec_YJ_transform,
              column="Yeo-Johnson")

The Yeo-Johnson transformation has the most effect on the data and therefore it will be used.

In [ ]:
df_prep.dataframe["total_rec_int"] = total_rec_YJ_transform

##### Delinq 2yrs

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="delinq_2yrs",
                title= "The number of 30+ days past-due payments in the borrower's credit file for the past 2 years",
                xlabel="Days",
                ylabel="Frequency")

The histogram show postive/right skew.

In [ ]:
delinq_log_transform = df_prep.skew_transform(df_prep.dataframe["delinq_2yrs"], transformation="log")
delinq_YJ_transform = df_prep.skew_transform(df_prep.dataframe["delinq_2yrs"], transformation="YJ")

plots.hist_plot(dataframe= delinq_log_transform,
                column="log",
                title= "Log transform number of 30+ days past-due payments in the borrower's credit file for the past 2 years",
                xlabel="Days",
                ylabel="Frequency")

plots.hist_plot(dataframe= delinq_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson transform number of 30+ days past-due payments in the borrower's credit file for the past 2 years",
                xlabel="Days",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=delinq_log_transform,
              column="log")
plots.plot_qq(dataframe=delinq_YJ_transform ,
              column="Yeo-Johnson")

In [ ]:
df_prep.dataframe["delinq_2yrs"]  = delinq_log_transform 

##### Out prncp

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="out_prncp",
                title= "Remaining outstanding principal for total amount funded",
                xlabel="Amount in pounds",
                ylabel="Frequency")

The histogram shows positive/right skew.

In [ ]:
# Apply log transformation.
out_pr_in_log_transform = df_prep.skew_transform(df_prep.dataframe["out_prncp"], transformation="log")
out_pr_YJ_transform = df_prep.skew_transform(df_prep.dataframe["out_prncp"], transformation="YJ")

# Plot log transformation
plots.hist_plot(dataframe= out_pr_in_log_transform,
                column="log",
                title= "Log transformed remaining outstanding principal for total amount funded",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Plot Box-Cox Yeo-Johnson
plots.hist_plot(dataframe = out_pr_YJ_transform,
                column="Yeo-Johnson",
                bins_size= 10, 
                title= "Yeo-Johnson transformed remaining outstanding principal for total amount funded", 
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=out_pr_in_log_transform,
              column="log")
plots.plot_qq(dataframe=out_pr_YJ_transform,
              column="Yeo-Johnson")

Histograms and qq plots showed that the transformation do not have great affect in normalising the data. Since the log transformation is the closet to normal distribution between the I will proceed wiht this transformation. However, this data  will require another transformation that brings the data closer to a normal distirbution.

In [ ]:
df_prep.dataframe["out_prncp"] = out_pr_in_log_transform


In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="out_prncp_inv",
                title= "Remaining outstanding principal for portion of total amount funded by investors",
                xlabel="Amount in pounds",
                ylabel="Frequency")

This histogram showed positve right skew.

In [ ]:
# Apply log transformation.
out_in_in_log_transform = df_prep.skew_transform(df_prep.dataframe["out_prncp_inv"], transformation="log")
out_in_YJ_transform = df_prep.skew_transform(df_prep.dataframe["out_prncp_inv"], transformation="YJ")

# Log transformation
plots.hist_plot(dataframe= out_in_in_log_transform,
                column="log",
                title= "Remaining outstanding principal for portion of total amount funded by investors",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Log transformation
plots.hist_plot(dataframe= out_in_YJ_transform,
                column="Yeo-Johnson",
                title= "Remaining outstanding principal for portion of total amount funded by investors",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=out_in_in_log_transform,
              column="log")
plots.plot_qq(dataframe=out_in_YJ_transform,
              column="Yeo-Johnson")

The histograms and qq plots showed that either transformation cannot fully correct the skew of the data. However, since the distribution of the log transform data is closer to a normal distribtuion than the Yeo-Johnson, this one will be use for this data.

In [ ]:
df_prep.dataframe["out_prncp_inv"] = out_in_in_log_transform 

##### Recoveries

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="recoveries",
                title= "Post charge off gross recovery",
                xlabel="Amount in pounds",
                ylabel="Frequency")

The histogram shows, positive/ rigth skew.

In [ ]:
# Apply log transformation.
recover_in_log_transform = df_prep.skew_transform(df_prep.dataframe["recoveries"], transformation="log")
recover_YJ_transform = df_prep.skew_transform(df_prep.dataframe["recoveries"], transformation="YJ")

# Log transformation
plots.hist_plot(dataframe= recover_in_log_transform,
                column="log",
                title= "Log transformed post charge off gross recovery",
                xlabel="Amount in pounds",
                ylabel="Frequency")

# Log transformation
plots.hist_plot(dataframe= recover_YJ_transform,
                column="Yeo-Johnson",
                title= " Yeo-Johnson transformed post charge off gross recovery",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=recover_in_log_transform,
              column="log")
plots.plot_qq(dataframe=recover_YJ_transform,
              column="Yeo-Johnson")

The histograms and qq plots showed that either transformation cannot fully correct the skew of the data. However, since the distribution of the log transform data is closer to a normal distribtuion than the Yeo-Johnson, this one will be use for this data.

In [ ]:
df_prep.dataframe["recoveries"] = recover_in_log_transform

##### Collection recovery fee

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="collection_recovery_fee", 
                title= "Post charge off collection fee",
                xlabel="Amount in pounds",
                ylabel="Frequency",
                bins_size=5)

In [ ]:
# Apply log transformation.
collection_in_log_transform = df_prep.skew_transform(df_prep.dataframe["collection_recovery_fee"], transformation="log")
collection_YJ_transform = df_prep.skew_transform(df_prep.dataframe["collection_recovery_fee"], transformation="YJ")

# Plot log transformation
plots.hist_plot(dataframe= collection_in_log_transform,
                column="log", 
                title= "Log transform post charge off collection fee",
                xlabel="Amount in pounds",
                ylabel="Frequency",
                bins_size=5)

# Plot Box-Cox transformation
plots.hist_plot(dataframe = collection_YJ_transform,
                 column="Yeo-Johnson",
                 title= "Yeo-Johnson transformed post charge off collection fee",
                 xlabel="Amount in pounds",
                 ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=collection_in_log_transform,
              column="log")
plots.plot_qq(dataframe=collection_YJ_transform,
               column="Yeo-Johnson")

The histograms and qqplots showed that the log transformation has the greates effect in correcting the skenes and therefore it will be used.

In [ ]:
df_prep.dataframe["collection_recovery_fee"] = collection_in_log_transform 

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="total_rec_late_fee",
                title= "Late fees received to date",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
# Apply log transformation.
total_p_in_log_transform = df_prep.skew_transform(df_prep.dataframe["total_rec_late_fee"], transformation="log")
total_p_YJ_transform = df_prep.skew_transform(df_prep.dataframe["total_rec_late_fee"], transformation="YJ")

plots.hist_plot(dataframe= total_p_in_log_transform,
                column="log",
                title= "Log transformed late fees received to date",
                xlabel="Amount in pounds",
                ylabel="Frequency")

plots.hist_plot(dataframe= total_p_YJ_transform,
                 column="Yeo-Johnson",
                 title= "Yeo-Johnson transformed late fees received to date",
                 xlabel="Amount in pounds",
                 ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=total_p_in_log_transform,
              column="log")
plots.plot_qq(dataframe=total_p_YJ_transform,
               column="Yeo-Johnson")

The histograms and qqplots showed that the log transformation has the greates effect in correcting the skenes and therefore it will be used.

In [ ]:
df_prep.dataframe["total_rec_late_fee"] = total_p_in_log_transform

##### Inq last 6mths

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="inq_last_6mths",
                title= "The number of inquiries in past 6 months (excluding auto and mortgage inquiries)",
                xlabel="Inquiries",
                ylabel="Frequency")

This histogram showed positive/right skew.

In [ ]:
# Apply log transformation.
inq_last_log_transform = df_prep.skew_transform(df_prep.dataframe["inq_last_6mths"], transformation="log")
inq_last_YJ_transform = df_prep.skew_transform(df_prep.dataframe["inq_last_6mths"], transformation="YJ")

# Plot log transformation
plots.hist_plot(dataframe = inq_last_log_transform ,
                column="log",
                title= "Log transformed number of inquiries in past 6 months (excluding auto and mortgage inquiries)",
                xlabel="inquiries",
                ylabel="Frequency")



# Plot Box-Cox transformation
plots.hist_plot(dataframe = inq_last_YJ_transform,
                 column="Yeo-Johnson",
                 title= "Yeo-Johnson transformed number of inquiries in past 6 months (excluding auto and mortgage inquiries)",
                 xlabel="inquiries",
                 ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=inq_last_log_transform,
              column="log")
plots.plot_qq(inq_last_YJ_transform,
               column="Yeo-Johnson")

The histograms and qqplots showed that the log transformation has the greates effect in correcting the skenes and therefore it will be used.

In [ ]:
df_prep.dataframe["inq_last_6mths"] = inq_last_log_transform

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="open_accounts",
                title= "The number of open credit lines in the borrower's credit file",
                xlabel="Open accounts",
                ylabel="Frequency")

In [ ]:
# Apply log transformation.
open_a_log_transform = df_prep.skew_transform(df_prep.dataframe["open_accounts"], transformation="log")
open_a_BC_transform = df_prep.skew_transform(df_prep.dataframe["open_accounts"], transformation="BC")
open_a_YJ_transform = df_prep.skew_transform(df_prep.dataframe["open_accounts"], transformation="YJ")

# Log transformation
plots.hist_plot(dataframe= open_a_log_transform,
                column="log",
                title= "Log transformed number of open credit lines in the borrower's credit file",
                xlabel="Open accounts",
                ylabel="Frequency")


# Box-Cox transformation
plots.hist_plot(dataframe=open_a_BC_transform,
                column="Box-Cox",
                title= "Box-Cox transformed number of open credit lines in the borrower's credit file",
                xlabel="Open accounts",
                ylabel="Frequency")

# Plot Box-Cox transformation
plots.hist_plot(dataframe = open_a_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson transformed number of open credit lines in the borrower's credit file",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=open_a_log_transform,
              column="log")
plots.plot_qq(dataframe=open_a_BC_transform,
              column="Box-Cox")
plots.plot_qq(dataframe=open_a_YJ_transform,
              column="Yeo-Johnson")

The histograms and qqplots showed that the Box-Cox transformation had has the greates effect in bringign the distribution closer to normal. Although the the difference is not much greater than the effect of Yeo-Johnson transformation. Therefore the Box-Cox transformation will be used.

In [ ]:
df_prep.dataframe["open_accounts"] = open_a_BC_transform


##### Total accounts

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="total_accounts",
                title= "The total number of credit lines currently in the borrower's credit file",
                xlabel="Number of accounts",
                ylabel="Frequency")

Positive/right skew

In [ ]:
total_ac_in_log_transform = df_prep.skew_transform(df_prep.dataframe["total_accounts"], transformation="log")
total_ac_BC_transform = df_prep.skew_transform(df_prep.dataframe["total_accounts"], transformation="BC")
total_ac_YJ_transform = df_prep.skew_transform(df_prep.dataframe["total_accounts"], transformation="YJ")

plots.hist_plot(dataframe= total_ac_in_log_transform,
                column="log",
                title= "Log transformed total number of credit lines currently in the borrower's credit file",
                xlabel="Number of accounts",
                ylabel="Frequency")

plots.hist_plot(dataframe= total_ac_BC_transform,
                column="Box-Cox",
                title= "Box-Cox transformed total number of credit lines currently in the borrower's credit file",
                xlabel="Number of accounts",
                ylabel="Frequency")

plots.hist_plot(dataframe= total_ac_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson transformed total number of credit lines currently in the borrower's credit file",
                xlabel="Number of accounts",
                ylabel="Frequency")



In [ ]:
plots.plot_qq(dataframe=total_ac_in_log_transform,
              column="log")
plots.plot_qq(dataframe=total_ac_BC_transform,
              column="Box-Cox")
plots.plot_qq(dataframe=total_ac_YJ_transform,
              column="Yeo-Johnson")

The histograms and qq plots shows that the box-Cox and Yeo-Johnson have the greates effect on the data. Since the data originally did not have any zero or negative values, the Box-Cox transformation will be used.

In [ ]:
df_prep.dataframe["total_accounts"] = total_ac_BC_transform 

##### Last payment amount

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="last_payment_amount",
                title= "Last total payment amount received",
                xlabel="Amount in pounds",
                ylabel="Frequency")


The histogram shows positive/right skew.

In [ ]:
# Apply transformations
last_p_in_log_transform = df_prep.skew_transform(df_prep.dataframe["last_payment_amount"], transformation="log")
last_p_YJ_transform = df_prep.skew_transform(df_prep.dataframe["last_payment_amount"], transformation="YJ")


plots.hist_plot(dataframe= last_p_in_log_transform,
                column="log",
                title= "Log trasnformed last total payment amount received",
                xlabel="Amount in pounds",
                ylabel="Frequency")

plots.hist_plot(dataframe= last_p_YJ_transform,
                column="Yeo-Johnson",
                title= "Yeo-Johnson trasnformed last total payment amount received",
                xlabel="Amount in pounds",
                ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=last_p_in_log_transform,
              column="log")
plots.plot_qq(dataframe=last_p_YJ_transform,
              column="Yeo-Johnson")

The histograms and qq plots show that the Yeo-Johnson transformation ahs the greates effect. Therefore it will be applied to the data.

In [ ]:
df_prep.dataframe["last_payment_amount"] = last_p_YJ_transform

##### Collections 12 mths ex med

In [ ]:
plots.hist_plot(dataframe= df_prep.dataframe,
                column="collections_12_mths_ex_med",
                title= "Number of collections in 12 months' excluding medical collections",
                xlabel="Collections",
                ylabel="Frequency")

In [ ]:
# Apply log transformation.
collections_log_transform = df_prep.skew_transform(df_prep.dataframe["collections_12_mths_ex_med"], transformation="log")
collections_YJ_transform = df_prep.skew_transform(df_prep.dataframe["collections_12_mths_ex_med"], transformation="YJ")

plots.hist_plot(dataframe= collections_log_transform,
                column="log",
                title= " Log transformed number of collections in 12 months' excluding medical collections",
                xlabel="Collections",
                ylabel="Frequency")

plots.hist_plot(dataframe= collections_YJ_transform,
                 column="Yeo-Johnson",
                 title= "Log transformed number of collections in 12 months' excluding medical collections",
                 xlabel="Collections",
                 ylabel="Frequency")

In [ ]:
plots.plot_qq(dataframe=collections_log_transform,
              column="log")
plots.plot_qq(dataframe=collections_YJ_transform,
               column="Yeo-Johnson")

The histograms and qq plots show that the log transformation has the greates effect. Therefore it will be use to transform the data.

In [ ]:
df_prep.dataframe["collections_12_mths_ex_med"] = collections_log_transform

# Outliers

In this sections numeric type columsn were visulised or tested for outliers wich then they were either removed or left.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="loan_amount", 
               title="Number of 30+ days past-due payments in the borrower's credit file for the past 2 years",
               xlabel="Number of 30+ days")

The histogram shows there is an otulier at the left wisker of the boxplot. However, since is close to the wisker it dosen't seem likely that it is an error.

In [ ]:
# z scores, the cutts of to be consider and outlier is normally >=  2 or 3
df_z = df_inf.z_score(dataframe=df_prep.dataframe, column="loan_amount")
# Turn on filter based on z_scores
df_z_cutoff = df_inf.z_score(dataframe=df_prep.dataframe, column="loan_amount", filter=True)
df_z_cutoff

The z_scores showed that 23875 are considered and outlier, being z_score > 2. However, removing that many data points will affect all the distribution distribution for other columns. 

In [ ]:
# Idetify outlier based on IQR
df_inf.IQR_filter_outliers(column="loan_amount", dataframe=df_z)

The interquartile range showed six data points with the same value, which means that the single outlier in the boxplot is actually six overlapping data points. Since all six outliers share the same value and are very close to the left wisker, they don't seem to be an error and therefore, they will remian in the column. The box-plot and interquantile range seem to less stringet than z_scores. Therefore both of them will be use for the subsequent columns.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="funded_amount", 
               title="Total amount committed to the loan at that point in time",
               xlabel="Amount in pounds")

# Idetify outlier based on IQR
df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="funded_amount")

The histogram shows there are 2 otuliers at the left wisker of the boxplot, which slightly overlap with each other.However, the interquartile range shows that there are 6 outliers, of which five of them have the same values. Since 5 of the outliers share the same value and overlap wiht the 6th outlier all of them close to the left wisker of the boxplot, they don't seem to be an error. Therefore, they will remian in the column.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="funded_amount_inv", 
               title="The total amount committed by investors for that loan at that point in time",
               xlabel="Amount commited by investors in pounds")

# Idetify outlier based on IQR
df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="funded_amount_inv")

The histogram and interquartile range showed 155 outliers near by the left wisker of the box plot. These most of which overlap with each other, except those wiht a zero value. Altough they are outisde of the wiskers, their clustering and overlapping suggest that these data points are no an error. Interially the data points wiht a zero value can be deleted but these can referring to loans funded by other means rather than investors. Taking into account that the outliers don't seem to be extreamily large, they will remain in the data.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="int_rate", 
               title="Annual interest rate of the loan",
               xlabel="Interest rates")

# Idetify outlier based on IQR
df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="int_rate")

The histogram and interquartile range showed 919 outliers near by the right wisker of the box plot. These most of which overlap with each other. Altough they are outisde of the wiskers, their clustering and overlapping suggest that these data points are no an error. To determine if these are left or remove the interest rate for their corresponding need to be check with the data for those years. Hoever, since their values son't seem to be very large they will remain in the data.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="instalment", 
               title="The monthly payment owned by the borrower, inclusive of the interest.",
               xlabel="Number of monthly paymets")

# Idetify outlier based on IQR
df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="instalment")

The histogram and interquartile range showed 41 outliers near by the right wisker of the box plot. These most of which overlap with each other. Altough they are outisde of the wiskers, their clustering and overlapping suggest that these data points are no an error. To determine if these are left or remove the interest rate for their corresponding need to be check with the data for those years. Hoever, since their values son't seem to be very large they will remain in the data.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="annual_inc", 
               title="The annual income of the borrower.",
               xlabel="Annual income.")

# Idetify outlier based on IQR
df_filter_outliers_an_in = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="annual_inc")
df_filter_outliers_an_in

The histogram and IQR_filter_outliers method shows 1335 ouliers. Altough there is a lot of overlapping and these data points don't seem to be an error, some of them have little over lapping. Therefore, we will remove all valu below 1.90, which are the values that less overlap between each other.

In [ ]:
outlier_delete_index_an_in = df_filter_outliers_an_in[df_filter_outliers_an_in["annual_inc"] < 1.90].index 
df_prep.remove_outlier(outlier_delete_index_an_in)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="annual_inc", 
               title="Outliers removed annual income of the borrower.",
               xlabel="Annual income.")


In [ ]:
df_filter_outliers_is_da = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="issue_date")
df_filter_outliers_is_da

The IQR_filter_outliers shows 23 outliers. However, since the issue dat for loan can depend in several factor, these data points will remain.

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="dti", 
               title="DTI ratio",
               xlabel="Ratio.")

# Idetify outlier based on IQR
df_filter_outliers_dti = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="dti")
df_filter_outliers_dti

The boxplot and QR_filter_outliers method shows 60 outliers close by the right wisker of the boxplot. Since these data points are overlapping and close to the wiskers, it suggest they are not an error and therefore  they will remain in the data.

In [ ]:
# Idetify outlier based on IQR
df_filter_outliers_ear_credit = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="earliest_credit_line")
df_filter_outliers_ear_credit

The QR_filter_outliers showed 1833 outliers. However, since dates can be affected by many  other variables, therefore these they will remain.

In [ ]:
plots.box_plot(dataframe=df_pre_im.dataframe, 
               column="open_accounts", 
               title="The number of open credit lines in the borrower's credit file",
               xlabel="number of open credit lines")

# Idetify outlier based on IQR
df_filter_outliers_op = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="open_accounts")
df_filter_outliers_op

The boxplot and QR_filter_outliers method show 693 outliers, 2 data point clusters the left wisker and 21 data clusters in the right wisker. The outliers on the left wisker will be remove and any putlier above 7.2 in the right wisker, since the remaining data point are very close to each other.

In [ ]:
outlier_delete_index_op = df_filter_outliers_op[(df_filter_outliers_op["open_accounts"] < 1) | (df_filter_outliers_op["open_accounts"] > 7.2)].index 
df_prep.remove_outlier(outlier_delete_index_op)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="open_accounts", 
               title="The number of open credit lines in the borrower's credit file",
               xlabel="number of open credit lines")

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_accounts", 
               title="total number of credit lines currently in the borrower's credit file",
               xlabel="Number of credit lines.")

# Idetify outlier based on IQR
df_filter_outliers_t_a = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="total_accounts")
df_filter_outliers_t_a

The boxplot and QR_filter_outliers method show 132 outliers, 1 data point clusters the left wisker and several others in the right wisker. The outliers on the left wisker will be remove and any putlier above 12.2 in the right wisker, since the the outliers begin to separate previous overlapping outliers.

In [ ]:
outlier_delete_index_t_a = df_filter_outliers_t_a[(df_filter_outliers_t_a["total_accounts"] < 2.3) | (df_filter_outliers_t_a["total_accounts"] > 12.2)].index 
df_prep.remove_outlier(outlier_delete_index_t_a)

In [ ]:
plots.box_plot(dataframe=df_prep, 
               column="total_accounts", 
               title="total number of credit lines currently in the borrower's credit file",
               xlabel="Number of credit lines")

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_payment", 
               title="Payments received to date for total amount funded",
               xlabel="Payments recieved.")

# Idetify outlier based on IQR
df_filter_outliers_t_p = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="total_payment")
df_filter_outliers_t_p

The boxplot and QR_filter_outliers method show 192 outliers with 3 clusters on the left wisker and two clustes in the right wisker. For both wisker  the outliers that are separating or in their own cluster away from the clusters touching the wisker will be be remove. 

In [ ]:
outlier_delete_index_t_p = df_filter_outliers_t_p[(df_filter_outliers_t_p["total_payment"] < 15) | (df_filter_outliers_t_p["total_payment"] > 97)].index 
df_prep.remove_outlier(outlier_delete_index_t_p)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_payment", 
               title="Payments received to date for total amount funded",
               xlabel="Payments recieved.")

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_payment_inv", 
               title="Payments received to date for total amount funded by investores",
               xlabel="Payments recieved.")

# Idetify outlier based on IQR
df_filter_outliers_t_i = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="total_payment_inv")
df_filter_outliers_t_i

The boxplot and QR_filter_outliers method show 1711 outliers with 1 main clustes in the right wisker. Outlier above 47000 will be remove, which are the outliers that are separting from the main cluster.

In [ ]:
outlier_delete_index_t_i = df_filter_outliers_t_i[df_filter_outliers_t_i["total_payment_inv"] > 47000].index 
df_prep.remove_outlier(outlier_delete_index_t_i)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_payment_inv", 
               title="Payments received to date for total amount funded by investores",
               xlabel="Payments recieved.")

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_rec_prncp", 
               title="Total recieved principal for total amount funded",
               xlabel="Amount recive")

# Idetify outlier based on IQR
df_filter_outliers_t_rec_pr = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="total_rec_prncp")
df_filter_outliers_t_rec_pr

The boxplot and QR_filter_outliers method show 2249 outliers with 1 main clustes in the right wisker. Outlier above 33000 will be remove, which are the outliers that are separting from the main cluster.

In [ ]:
outlier_delete_index_t_rec_pr = df_filter_outliers_t_rec_pr[df_filter_outliers_t_rec_pr["total_rec_prncp"] > 30200].index 
df_prep.remove_outlier(outlier_delete_index_t_rec_pr)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_rec_prncp", 
               title="Total recieved principal for total amount funded",
               xlabel="Amount recive")

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_rec_int", 
               title="Interest received to date",
               xlabel="Interest")

# Idetify outlier based on IQR
df_filter_outliers_t_r_i = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="total_rec_int")
df_filter_outliers_t_r_i

The boxplot and QR_filter_outliers method show 2249 outliers with 2 main clustes in the right wisker and 1 main cluster in the right wisker. Outlier less than 4 and above 28 will be remove, which are the outliers that are separting from the main cluster.

In [ ]:
outlier_delete_index_t_r_i = df_filter_outliers_t_r_i[(df_filter_outliers_t_r_i["total_rec_int"] < 4) | (df_filter_outliers_t_r_i["total_rec_int"] > 28) ].index 
df_prep.remove_outlier(outlier_delete_index_t_r_i)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="total_rec_int", 
               title="Interest received to date",
               xlabel="Interest")

In [ ]:
# Idetify outlier based on IQR
df_filter_outliers_l_p_d= df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="last_payment_date")
df_filter_outliers_l_p_d

QR_filter_outliers method show 1069 outliers. However since dates can be affected by several other variables, the data will remain.

In [ ]:

plots.box_plot(dataframe=df_prep.dataframe, 
               column="last_payment_amount", 
               title="Last total payment amount received",
               xlabel="Payment amount")

# Idetify outlier based on IQR
df_filter_outliers_l_p_a = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="last_payment_amount")
df_filter_outliers_l_p_a

The boxplot and QR_filter_outliers method show 223 outliers with twom main cluster data points. Outliers less than 1.7 since from that values data points begin to separate from the main cluster touchin the wisker.

In [ ]:
outlier_delete_index_l_p_a = df_filter_outliers_l_p_a[df_filter_outliers_l_p_a["last_payment_amount"] < 1.7].index 
df_prep.remove_outlier(outlier_delete_index_l_p_a)

In [ ]:
plots.box_plot(dataframe=df_prep.dataframe, 
               column="last_payment_amount", 
               title="Last total payment amount received",
               xlabel="Payment amount")

In [ ]:
# Idetify outlier based on IQR
df_filter_outliers_n_p_d = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="next_payment_date")
df_filter_outliers_n_p_d

Since dates can be infleunce by several other factors, this data will remain.

In [ ]:
# Idetify outlier based on IQR
df_filter_outliers_l_c_p = df_inf.IQR_filter_outliers(dataframe=df_prep.dataframe, column="last_credit_pull_date")
df_filter_outliers_l_c_p

Since dates can be infleunce by several other factors, this data will remain.

# Correlation


Highly correlated column can predict with very high degree of accuracy, and thus making the dataframe more compact. Thios section aim tocheck  the correlation between the columns of the data frame and then select wether to remove or leave tose columns in the data.

The correlation matrix will be compute for the whole data frame to idetify possible correlating columns.

In [ ]:
sns.pairplot(df_prep.dataframe)
plt.show()

From the above figure correlation between varibales of the dataframe can be seen predominately positive correlation across the dataset, such as:


funded amount:
- total_account
- out_prncp 
- total_rec_late_fee linear
- recoveries 
- collection_fee_linear

fundedamount_inv:
- out_prncp 
- out_prncp_inv


instalmest:
- loan_amount
- annual_inc 
- out_prcncp 
- out_prcncp_inv 
- total_payment 
- total_payment_inv 
- total_rec_prcnp 
- total_rec_int 
	
annual_inc
- loan_amount 
- instalment 
- ut_prcncp 
- out_prcncp_inv 
- total_payment_inv 
- total_rec_prcnp 
- total_rec_int 

To better visualise the correlation between the variables in the dataset a heat map was created using pearson's correalation with a threshold value of 0.90. This satatictic was chosen because measures linear relationships and is sensitive to both magnitud and direction of the linear association. In addtion the data has been normalize and the outliers were dealt with. However, since all the outliers were deleted Kendall and Spearman could also be used since they are more robust and make fewer assumptions about the distribution of the data.

In [ ]:
# Calculate correlation matrix
corr = df_prep.dataframe.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(14, 12))

# Generate  a palette
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Generate heatmap
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

The above figure show pearson's correalation predominately positive correlation in the dataset, particularly with:
- loan_amount
- funded_amount
- funded_amount_inv
- instalment
- annual_inc
- out_prncp
- out_prncp
- total_payment
- total_payment_inv
- total_rec_prncp
- total_rec_int
- total_rec_late_fee
- recoveries
- Collection_recovery_fee

Interestintly the heatmap showed no correlation between recovery, collection_recovery_fee and out_prncp and out_prncp_inv


Based on the threhold value of 0.90 these are the columns correlated:
funded_amount: loan_amount

funded_amount_inv: loan_amount, funded_amount

Instalment: loan_amount

annual_inc: loan_amount, instalment, funded_amount_inv

total_payment_inv: total_payment

total_rec_prncp: total_payment, total_payment_inv


With regards to funded_amount, loan_amount, funded_amount_inv, instalmets and annual_inc. They reflect different importan aspects of the loan. Deleting them will hinder the interpredation adn further analysis of the data. Therefore these columns will remian.

Since total_rec_prncp is correlated to total_payment, total_payment_inv, which are more important for inepretation, total_rec_prncp could be removed. Therefore,  two linear regresion models will be created and based on the Variance Inflation Factor (IVF) values one of the columns will be removed. VIF is greater than 1 shows the presence of multicollinearity and therefore one of the columns will be remove.

In [ ]:
# Fit a linear regression model to try and predict total_rec_prncp 
t_r_p_model = smf.ols("total_rec_prncp ~ total_payment + total_payment_inv", df_prep.dataframe).fit()
t_p_model = smf.ols("total_payment ~ total_rec_prncp + total_payment_inv", df_prep.dataframe).fit()
t_p_i_model = smf.ols("total_payment_inv  ~ total_rec_prncp + total_payment", df_prep.dataframe).fit()

# Calculate Variance Inflation factor
VIF_t_r_p_model = round(1/(1-t_r_p_model.rsquared), 2)
VIF_t_p_model = round(1/(1-t_p_model.rsquared), 2)
VIF_t_p_i_model = round(1/(1-t_p_i_model.rsquared), 2)

#Print results
print(f"VIF scores: \n total_rec_prncp: {VIF_t_r_p_model} \n total_payment: {VIF_t_p_model} \n total_payment_inv: {VIF_t_p_i_model}")

The VIF values shows the presence of multicollinearity between total_rec_prncp, total_payment and total_payment_inv with the model predicting the total_payment_inv being the highest. However, total_payment and total_payment_inv are more helpful at intepreting the dataset than total_rec_prncp. Therefore that column will be removed.

In [ ]:
df_prep.remove_columns("total_rec_prncp")